In [1]:
%load_ext autoreload
%autoreload 2

import pickle 

def to_pickle(df, f):
    with open(f, 'wb') as fname:
        pickle.dump(df, fname)

def open_pickle(f):
    with open(f, 'rb') as file:
        data = pickle.load(file)
    return data  

In [2]:
import pandas as pd 
from transformers import LlamaConfig
import torch 
import transformers
from transformers import AutoTokenizer

#from custom_model_mlp import CustomModel
from custom_model import CustomModel

cui_embeds = open_pickle("/n/home01/ruthjohnson/kg_paper_revision/model/umls_cui_embeds.pkl")
node_df = pd.read_csv("/n/home01/ruthjohnson/kg_paper_revision/connected_node_v2_df.csv", sep='\t')
keep_cui = set(node_df.loc[node_df['ntype'] == 'UMLS_CUI']['node_id'].str.split(':', expand=True)[0].tolist())
keep_cui = list(keep_cui)

tokenizer = AutoTokenizer.from_pretrained(
    "Henrychur/MMed-Llama-3-8B-EnIns",
    model_max_length=2048,
    use_fast=False,
    trust_remote_code=True
)
tokenizer.pad_token = tokenizer.eos_token

cui_tokens = [('<%s>' % x) for x in list(keep_cui)]
tokenizer.add_tokens(cui_tokens, special_tokens=True)

/n/home01/ruthjohnson/.local/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/n/home01/ruthjohnson/.local/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/n/home01/ruthjohnson/.local/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 t

118371

In [3]:
with torch.no_grad():
    config = LlamaConfig.from_pretrained("Henrychur/MMed-Llama-3-8B-EnIns")
    config.architectures = ["CustomModel"]

    lin_model = CustomModel(config=config).from_pretrained("Henrychur/MMed-Llama-3-8B-EnIns")
    #model.resize_token_embeddings(len(tokenizer))
    lin_model.setup(cui_embeds)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

Some weights of CustomModel were not initialized from the model checkpoint at Henrychur/MMed-Llama-3-8B-EnIns and are newly initialized: ['proj_layer.bias', 'proj_layer.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/n/holylfs06/LABS/mzitnik_lab/Lab/ruthjohnson/kg_paper_revision/medqa/custom_model.py:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.cui_embeds = torch.tensor(cui_embeds, requires_grad=True)


In [4]:
import transformers
from train import ModelArguments, DataArguments, TrainingArguments

# Define simulated command-line arguments
args_list = [
    "--model_name_or_path", "Henrychur/MMed-Llama-3-8B-EnIns",
    "--data_path", "/n/holylfs06/LABS/mzitnik_lab/Lab/ruthjohnson/kg_paper_revision/medqa/MMedBench/Train_eng",
    "--output_dir", "/n/home01/ruthjohnson/ruthjohnson/kg_paper_revision/medqa/ft_plus_linear_results",    
    "--bf16", "True",
    "--num_train_epochs", "3",
    "--per_device_train_batch_size", "1",
    "--gradient_accumulation_steps", "10",
    "--evaluation_strategy", "no",
    "--save_strategy", "epoch", 
    "--save_total_limit", "3",
    "--learning_rate", "2e-5",
    "--weight_decay", "0.",
    "--warmup_ratio", "0.03",
    "--lr_scheduler_type", "cosine",
    "--logging_steps", "1",
    "--is_lora", "True",
    "--local_rank", "16",
    "--target_modules", "q_proj", "v_proj", "proj_layer"
    #, "proj_layer"
]
# "proj_layer_1", "proj_layer_2", "proj_layer_3", "proj_layer_4" 

# Initialize the HfArgumentParser
parser = transformers.HfArgumentParser((ModelArguments, DataArguments, TrainingArguments))

# Parse the simulated arguments into dataclasses
model_args, data_args, training_args = parser.parse_args_into_dataclasses(args_list)
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r = model_args.lora_rank,
    lora_alpha = 32, #model_args.lora_alpha,
    target_modules = model_args.target_modules,
    lora_dropout = 0.05,
    bias = 'none',
    task_type="CAUSAL_LM",
)

lin_model = get_peft_model(lin_model, config)

/n/home01/ruthjohnson/venv_dgl/lib/python3.10/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [5]:
with torch.no_grad():
    lin_model.proj_layer.load_state_dict(torch.load('/n/home01/ruthjohnson/ruthjohnson/kg_paper_revision/medqa/ft_linear/model_weights.pth', weights_only=True))

In [6]:
from peft import AutoPeftModel, AutoPeftModelForCausalLM

model = AutoPeftModelForCausalLM.from_pretrained("/n/holylfs06/LABS/mzitnik_lab/Lab/ruthjohnson/kg_paper_revision/medqa/ft_linear")

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [7]:
# get updated embeddings
orig_emb_data = model.get_input_embeddings().weight

#x = lin_model.proj_layer_1(lin_model.cui_embeds)
#x = lin_model.relu(lin_model.proj_layer_2(x))  # Second layer
#x = lin_model.relu(lin_model.proj_layer_3(x))  # Third layer
#new_token_data = lin_model.proj_layer_4(x) 
new_token_data = lin_model.proj_layer(lin_model.cui_embeds)

# append new token weights
new_emb_data = torch.cat((orig_emb_data.data, new_token_data))

# assign new embeddings
new_input_embedding = torch.nn.Embedding(new_emb_data.shape[0], new_emb_data.shape[1])
new_input_embedding.weight = torch.nn.Parameter(new_emb_data)
model.set_input_embeddings(new_input_embedding)

In [8]:
# non-cui model (still ft though)
basic_model = AutoPeftModelForCausalLM.from_pretrained("/n/holylfs06/LABS/mzitnik_lab/Lab/ruthjohnson/kg_paper_revision/medqa/ft_basic")

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

/n/home01/ruthjohnson/.local/lib/python3.10/site-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model.model.layers.1.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.1.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.1.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.layers.1.self_attn.v_proj.lora_B.default.weight', 'base_model.model.model.layers.2.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.2.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.2.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.layers.2.

In [17]:
from inference import prepare_data, inference_on_one

test_new_list_data_dict = open_pickle("train_list_data_dict.pkl")
data_list = prepare_data(test_new_list_data_dict)

100%|██████████| 10178/10178 [00:00<00:00, 577894.25it/s]


In [ ]:
response_list = []

idx = 0

for idx in range(0, 100):
    data_entry = data_list[idx]
    input_str = [
        data_entry['pmc_input']
    ]

    gt = data_entry['pmc_output']

    ######
    with torch.no_grad():
        # no cui
        str_list = input_str[0].split("<")
        basic_input_str = [str_list[0] + "\n\n### Response:"]
        output_str = inference_on_one(basic_input_str, model.cuda(), tokenizer)
        response = output_str.split("### Response:")[1].strip()
        basic_response = response.lower().split("###answer: option ")[1].split('###')[0][0:1].upper()

        #print('\n')
        #print(basic_response)

        # original cui
        output_str = inference_on_one(input_str, model.cuda(), tokenizer)
        response = output_str.split("### Response:")[1].strip()
        ft_response = response.lower().split("###answer: option ")[1].split('###')[0][0:1].upper()

        #print(ft_response)

        # additional cui
        #str_list = input_str[0].split("\n\n### Response:")
        #new_input_str = [str_list[0] + ' <C0022860>' + "\n\n### Response:"]

        #output_str = inference_on_one(new_input_str, model.cuda(), tokenizer)
        #response = output_str.split("### Response:")[1].strip()
        #print(response)
        #print(gt)
        response_list.append([ft_response, basic_response, gt])
        print([ft_response, basic_response, gt])


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


['D', 'D', 'D']


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


['A', 'A', 'A']


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


['A', 'A', 'A']


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


['A', 'C', 'C']


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [14]:
gt

'B'

In [12]:
basic_input_str

["Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nYou're a English doctor, kindly address the medical queries according to the patient's account. Answer with the best option directly.\n\n### Input:\n###Question: A 24-year-old G2P1 woman at 39 weeks’ gestation presents to the emergency department complaining of painful contractions occurring every 10 minutes for the past 2 hours, consistent with latent labor. She says she has not experienced vaginal discharge, bleeding, or fluid leakage, and is currently taking no medications. On physical examination, her blood pressure is 110/70 mm Hg, heart rate is 86/min, and temperature is 37.6°C (99.7°F). She has had little prenatal care and uses condoms inconsistently. Her sexually transmitted infections status is unknown. As part of the patient’s workup, she undergoes a series of rapid screening tests that result in t

In [ ]:
response = response.lower()
        try:
            response = response.split("###answer: option ")[1].split('###')[0][0:1].upper()